In [ ]:
initial_perm = [0x58, 0x50, 0x42, 0x34, 0x26, 0x18, 0x10, 0x2,
				0x60, 0x52, 0x44, 0x36, 0x28, 0x20, 0x12, 0x4,
				0x62, 0x54, 0x46, 0x38, 0x30, 0x22, 0x14, 0x6,
				0x64, 0x56, 0x48, 0x40, 0x32, 0x24, 0x16, 0x8,
				0x57, 0x49, 0x41, 0x33, 0x25, 0x17, 0x9, 0x1,
				0x59, 0x51, 0x43, 0x35, 0x27, 0x19, 0x11, 0x3,
				0x61, 0x53, 0x45, 0x37, 0x29, 0x21, 0x13, 0x5,
				0x63, 0x55, 0x47, 0x39, 0x31, 0x23, 0x15, 0x7]

exp_d = [0x32, 0x1, 0x2, 0x3, 0x4, 0x5, 0x4, 0x5,
		0x6, 0x7, 0x8, 0x9, 0x8, 0x9, 0x10, 0x11,
		0x12, 0x13, 0x12, 0x13, 0x14, 0x15, 0x16, 0x17,
		0x16, 0x17, 0x18, 0x19, 0x20, 0x21, 0x20, 0x21,
		0x22, 0x23, 0x24, 0x25, 0x24, 0x25, 0x26, 0x27,
		0x28, 0x29, 0x28, 0x29, 0x30, 0x31, 0x32, 0x1]

per = [0x16, 0x7, 0x20, 0x21,
	0x29, 0x12, 0x28, 0x17,
	0x1, 0x15, 0x23, 0x26,
	0x5, 0x18, 0x31, 0x10,
	0x2, 0x8, 0x24, 0x14,
	0x32, 0x27, 0x3, 0x9,
	0x19, 0x13, 0x30, 0x6,
	0x22, 0x11, 0x4, 0x25]

sbox = [[[0x14, 0x4, 0x13, 0x1, 0x2, 0x15, 0x11, 0x8, 0x3, 0x10, 0x6, 0x12, 0x5, 0x9, 0x0, 0x7],
		[0x0, 0x15, 0x7, 0x4, 0x14, 0x2, 0x13, 0x1, 0x10, 0x6, 0x12, 0x11, 0x9, 0x5, 0x3, 0x8],
		[0x4, 0x1, 0x14, 0x8, 0x13, 0x6, 0x2, 0x11, 0x15, 0x12, 0x9, 0x7, 0x3, 0x10, 0x5, 0x0],
		[0x15, 0x12, 0x8, 0x2, 0x4, 0x9, 0x1, 0x7, 0x5, 0x11, 0x3, 0x14, 0x10, 0x0, 0x6, 0x13]],

		[[0x15, 0x1, 0x8, 0x14, 0x6, 0x11, 0x3, 0x4, 0x9, 0x7, 0x2, 0x13, 0x12, 0x0, 0x5, 0x10],
		[0x3, 0x13, 0x4, 0x7, 0x15, 0x2, 0x8, 0x14, 0x12, 0x0, 0x1, 0x10, 0x6, 0x9, 0x11, 0x5],
		[0x0, 0x14, 0x7, 0x11, 0x10, 0x4, 0x13, 0x1, 0x5, 0x8, 0x12, 0x6, 0x9, 0x3, 0x2, 0x15],
		[0x13, 0x8, 0x10, 0x1, 0x3, 0x15, 0x4, 0x2, 0x11, 0x6, 0x7, 0x12, 0x0, 0x5, 0x14, 0x9]],

		[[0x10, 0x0, 0x9, 0x14, 0x6, 0x3, 0x15, 0x5, 0x1, 0x13, 0x12, 0x7, 0x11, 0x4, 0x2, 0x8],
		[0x13, 0x7, 0x0, 0x9, 0x3, 0x4, 0x6, 0x10, 0x2, 0x8, 0x5, 0x14, 0x12, 0x11, 0x15, 0x1],
		[0x13, 0x6, 0x4, 0x9, 0x8, 0x15, 0x3, 0x0, 0x11, 0x1, 0x2, 0x12, 0x5, 0x10, 0x14, 0x7],
		[0x1, 0x10, 0x13, 0x0, 0x6, 0x9, 0x8, 0x7, 0x4, 0x15, 0x14, 0x3, 0x11, 0x5, 0x2, 0x12]],

		[[0x7, 0x13, 0x14, 0x3, 0x0, 0x6, 0x9, 0x10, 0x1, 0x2, 0x8, 0x5, 0x11, 0x12, 0x4, 0x15],
		[0x13, 0x8, 0x11, 0x5, 0x6, 0x15, 0x0, 0x3, 0x4, 0x7, 0x2, 0x12, 0x1, 0x10, 0x14, 0x9],
		[0x10, 0x6, 0x9, 0x0, 0x12, 0x11, 0x7, 0x13, 0x15, 0x1, 0x3, 0x14, 0x5, 0x2, 0x8, 0x4],
		[0x3, 0x15, 0x0, 0x6, 0x10, 0x1, 0x13, 0x8, 0x9, 0x4, 0x5, 0x11, 0x12, 0x7, 0x2, 0x14]],

		[[0x2, 0x12, 0x4, 0x1, 0x7, 0x10, 0x11, 0x6, 0x8, 0x5, 0x3, 0x15, 0x13, 0x0, 0x14, 0x9],
		[0x14, 0x11, 0x2, 0x12, 0x4, 0x7, 0x13, 0x1, 0x5, 0x0, 0x15, 0x10, 0x3, 0x9, 0x8, 0x6],
		[0x4, 0x2, 0x1, 0x11, 0x10, 0x13, 0x7, 0x8, 0x15, 0x9, 0x12, 0x5, 0x6, 0x3, 0x0, 0x14],
		[0x11, 0x8, 0x12, 0x7, 0x1, 0x14, 0x2, 0x13, 0x6, 0x15, 0x0, 0x9, 0x10, 0x4, 0x5, 0x3]],

		[[0x12, 0x1, 0x10, 0x15, 0x9, 0x2, 0x6, 0x8, 0x0, 0x13, 0x3, 0x4, 0x14, 0x7, 0x5, 0x11],
		[0x10, 0x15, 0x4, 0x2, 0x7, 0x12, 0x9, 0x5, 0x6, 0x1, 0x13, 0x14, 0x0, 0x11, 0x3, 0x8],
		[0x9, 0x14, 0x15, 0x5, 0x2, 0x8, 0x12, 0x3, 0x7, 0x0, 0x4, 0x10, 0x1, 0x13, 0x11, 0x6],
		[0x4, 0x3, 0x2, 0x12, 0x9, 0x5, 0x15, 0x10, 0x11, 0x14, 0x1, 0x7, 0x6, 0x0, 0x8, 0x13]],

		[[0x4, 0x11, 0x2, 0x14, 0x15, 0x0, 0x8, 0x13, 0x3, 0x12, 0x9, 0x7, 0x5, 0x10, 0x6, 0x1],
		[0x13, 0x0, 0x11, 0x7, 0x4, 0x9, 0x1, 0x10, 0x14, 0x3, 0x5, 0x12, 0x2, 0x15, 0x8, 0x6],
		[0x1, 0x4, 0x11, 0x13, 0x12, 0x3, 0x7, 0x14, 0x10, 0x15, 0x6, 0x8, 0x0, 0x5, 0x9, 0x2],
		[0x6, 0x11, 0x13, 0x8, 0x1, 0x4, 0x10, 0x7, 0x9, 0x5, 0x0, 0x15, 0x14, 0x2, 0x3, 0x12]],

		[[0x13, 0x2, 0x8, 0x4, 0x6, 0x15, 0x11, 0x1, 0x10, 0x9, 0x3, 0x14, 0x5, 0x0, 0x12, 0x7],
		[0x1, 0x15, 0x13, 0x8, 0x10, 0x3, 0x7, 0x4, 0x12, 0x5, 0x6, 0x11, 0x0, 0x14, 0x9, 0x2],
		[0x7, 0x11, 0x4, 0x1, 0x9, 0x12, 0x14, 0x2, 0x0, 0x6, 0x10, 0x13, 0x15, 0x3, 0x5, 0x8],
		[0x2, 0x1, 0x14, 0x7, 0x4, 0x10, 0x8, 0x13, 0x15, 0x12, 0x9, 0x0, 0x3, 0x5, 0x6, 0x11]]]

# Final Permutation Table
final_perm = [0x40, 0x8, 0x48, 0x16, 0x56, 0x24, 0x64, 0x32,
			0x39, 0x7, 0x47, 0x15, 0x55, 0x23, 0x63, 0x31,
			0x38, 0x6, 0x46, 0x14, 0x54, 0x22, 0x62, 0x30,
			0x37, 0x5, 0x45, 0x13, 0x53, 0x21, 0x61, 0x29,
			0x36, 0x4, 0x44, 0x12, 0x52, 0x20, 0x60, 0x28,
			0x35, 0x3, 0x43, 0x11, 0x51, 0x19, 0x59, 0x27,
			0x34, 0x2, 0x42, 0x10, 0x50, 0x18, 0x58, 0x26,
			0x33, 0x1, 0x41, 0x9, 0x49, 0x17, 0x57, 0x25]

In [ ]:
def convert_bstring_to_bin(bstring):
  bin_str = ""
  for char in bstring:
    bin_str += bin(ord(char))[2:] + ' '
  return bin_str


print(convert_bstring_to_bin('1234'))
